# Data Preprocessing

In [1]:
import pandas as pd

# 데이터 파일 읽기
file_path = 'NVDA.csv'
data = pd.read_csv(file_path)

# 데이터 확인
print(data.head())

             날짜     종가     시가     고가     저가      거래량    변동 %
0  2021- 08- 23  21.96  20.97  22.00  20.95  575.81M   5.48%
1  2021- 08- 20  20.82  19.99  20.87  19.93  675.74M   5.15%
2  2021- 08- 19  19.80  19.49  20.50  18.76  766.55M   3.99%
3  2021- 08- 18  19.04  19.50  19.63  19.00  285.91M  -2.16%
4  2021- 08- 17  19.46  19.68  19.77  19.27  204.58M  -2.46%


# Env Setting

In [ ]:
import pandas as pd
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
from gym import spaces

class StockEnv(gym.Env):
    def __init__(self, df, N, M, news_events):
        super(StockEnv, self).__init__()
        self.df = df
        self.N = N
        self.M = M
        self.news_events = news_events
        self.current_step = 0
        self.action_space = spaces.Discrete(3)  # 0: 풀매수, 1: 풀매도, 2: 존버
        self.observation_space = spaces.Box(low=0, high=1, shape=(len(df.columns),), dtype=np.float32)
        
    def reset(self):
        self.current_step = len(self.df) - (self.N * 12 + self.M)  # 가장 최근 날짜로 설정
        return self._next_observation()
    
    def _next_observation(self):
        obs = self.df.iloc[self.current_step].values
        return obs
    
    def step(self, action):
        self.current_step += 1
        
        if self.current_step >= len(self.df) - 1:
            done = True
        else:
            done = False
        
        reward = self._calculate_reward(action)
        
        obs = self._next_observation()
        
        return obs, reward, done, {}
    
    def _calculate_reward(self, action):
        current_price = self.df.iloc[self.current_step]['종가']
        future_step = self.current_step + (self.N * 12 + self.M)  # N년 M개월 뒤의 스텝 계산
        if future_step >= len(self.df):
            future_price = self.df.iloc[-1]['종가']
        else:
            future_price = self.df.iloc[future_step]['종가']
        
        news = self.news_events[self.current_step]  # 현재 스텝의 뉴스 이벤트
        news_factor = 1.0
        if news == 'Good':
            news_factor = 1.8  # 좋은 뉴스일 경우 보상 증가 (1년동안 오른 정도 반영)
        elif news == 'Bad':
            news_factor = 0.7  # 나쁜 뉴스일 경우 보상 감소 (1달 전 최저가 반영)
        
        if action == 0:  # 풀매수 가즈아~
            reward = (future_price - current_price) * news_factor
        elif action == 1:  # 풀매도 가즈아!
            reward = (current_price - future_price) * news_factor
        else:  # 존~~~~~버
            reward = 0
        
        return reward
    
    def render(self, mode='human'):
        pass

In [ ]:

# Step 1: 데이터 파일 읽기
file_path = '/mnt/data/NVDA.csv'
data = pd.read_csv(file_path)

# 데이터 전처리
data = data[['날짜', '종가']]  # 필요한 컬럼만 선택
data['날짜'] = pd.to_datetime(data['날짜'])
data.set_index('날짜', inplace=True)

# 뉴스 이벤트 리스트 생성 (예시)
news_events = np.random.choice(['Good', 'Bad'], len(data))

# Step 2: 환경 설정
N = 2  # 년
M = 6  # 개월
env = DummyVecEnv([lambda: StockEnv(data, N, M, news_events)])

# Step 3: 에이전트 설정 및 학습
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

# Step 4: 평가
obs = env.reset()
for i in range(200):
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)
    env.render()

print("학습 완료")


# Step 3: 에이전트 설정 및 학습
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

# 학습 완료 후 모델 저장
model.save("ppo_stock_model")


In [32]:
import pandas as pd
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
from gym import spaces
from datetime import timedelta

class StockEnv(gym.Env):
    def __init__(self, df, N, M, news_events):
        super(StockEnv, self).__init__()
        self.df = df
        self.N = N
        self.M = M
        self.news_events = news_events
        self.current_step = len(df) - 1  # 가장 최근 날짜로 초기화
        self.action_space = spaces.Discrete(3)  # 0: 풀매수, 1: 풀매도, 2: 존버
        self.observation_space = spaces.Box(low=0, high=1, shape=(len(df.columns),), dtype=np.float32)
        
    def reset(self):
        self.current_step = len(self.df) - 1  # 가장 최근 날짜로 초기화
        return self._next_observation()
    
    def _next_observation(self):
        obs = self.df.iloc[self.current_step].values
        return obs
    
    def step(self, action):
        self.current_step -= 1  # 과거로 이동
        
        if self.current_step < 0:
            done = True
        else:
            done = False
        
        reward = self._calculate_reward(action)
        
        obs = self._next_observation()
        
        return obs, reward, done, {}
    
    def _calculate_reward(self, action):
        current_price = self.df.iloc[self.current_step]['종가']
        future_step = self.current_step - (self.N * 12 + self.M)  # N년 M개월 전의 스텝 계산
        if future_step < 0:
            future_price = self.df.iloc[0]['종가']
        else:
            future_price = self.df.iloc[future_step]['종가']
        
        news = self.news_events[self.current_step % len(self.news_events)]  # 현재 스텝의 뉴스 이벤트
        news_factor = 1.0
        if news == 'Good':
            news_factor = 1.8  # 좋은 뉴스일 경우 보상 증가 (1년동안 오른 정도 반영)
        elif news == 'Bad':
            news_factor = 0.7  # 나쁜 뉴스일 경우 보상 감소 (1달 전 최저가 반영)
        
        if action == 0:  # 풀매수 가즈아~
            reward = (future_price - current_price) * news_factor
        elif action == 1:  # 풀매도 가즈아!
            reward = (current_price - future_price) * news_factor
        else:  # 존~~~~~버
            reward = 0
        
        return reward
    
    def render(self, mode='human'):
        pass

# Step 1: 데이터 파일 읽기
file_path = 'NVDA.csv'
data = pd.read_csv(file_path)

# 데이터 전처리
data = data[['날짜', '종가']]  # 필요한 컬럼만 선택
data['날짜'] = pd.to_datetime(data['날짜'])
data.sort_values(by='날짜', ascending=True, inplace=True)  # 날짜를 기준으로 정렬
data.set_index('날짜', inplace=True)

# 사용자 입력으로 뉴스 이벤트 리스트 생성
user_news_events = ['Good', 'Good', 'Bad', 'Good', 'Good', 'Good', 'Good']  # 예시 입력
news_events = user_news_events * (len(data) // len(user_news_events)) + user_news_events[:len(data) % len(user_news_events)]

# Step 2: 환경 설정
N = 3  # 년
M = 6  # 개월

# 환경 생성
env = DummyVecEnv([lambda: StockEnv(data, N, M, news_events)])

# 학습된 모델 로드
model = PPO.load("ppo_stock_model.zip")  # 학습된 모델 파일 경로

# 특정 시점의 데이터로 환경 초기화
obs = env.reset()

# N년 M개월 동안의 예측
steps = N * 12 + M

predicted_prices = []
predicted_dates = []

# 시작 날짜
start_date = data.index[-1]

for _ in range(steps):
    action, _ = model.predict(obs)
    obs, rewards, dones, info = env.step([action])
    current_step = env.envs[0].current_step
    current_price = env.envs[0].df.iloc[current_step]['종가']
    current_date = start_date + timedelta(days=(steps - len(predicted_prices)) * 30)  # 매 스텝 30일 추가
    predicted_prices.append(current_price)
    predicted_dates.append(current_date)
    if dones:
        break

predicted_df = pd.DataFrame({
    '날짜': predicted_dates,
    '예측 종가': predicted_prices
})

# 표 형태로 출력
predicted_df = predicted_df.sort_values(by='날짜', ascending=True)  # 날짜 오름차순 정렬
print(predicted_df)

# 예측 결과 저장 (선택 사항)
predicted_df.to_csv('predicted_prices.csv', index=False)


           날짜  예측 종가
41 2021-09-22  19.06
40 2021-10-22  19.21
39 2021-11-21  19.03
38 2021-12-21  19.98
37 2022-01-20  20.03
36 2022-02-19  20.00
35 2022-03-21  20.21
34 2022-04-20  20.49
33 2022-05-20  20.70
32 2022-06-19  20.37
31 2022-07-19  19.90
30 2022-08-18  20.05
29 2022-09-17  20.51
28 2022-10-17  20.25
27 2022-11-16  19.84
26 2022-12-16  18.97
25 2023-01-15  18.16
24 2023-02-14  18.78
23 2023-03-16  18.61
22 2023-04-15  19.41
21 2023-05-15  19.59
20 2023-06-14  19.56
19 2023-07-14  19.29
18 2023-08-13  19.21
17 2023-09-12  19.50
16 2023-10-12  19.66
15 2023-11-11  19.50
14 2023-12-11  19.75
13 2024-01-10  19.82
12 2024-02-09  20.27
11 2024-03-10  20.64
10 2024-04-09  20.37
9  2024-05-09  20.30
8  2024-06-08  19.94
7  2024-07-08  19.70
6  2024-08-07  19.91
5  2024-09-06  20.19
4  2024-10-06  19.95
3  2024-11-05  19.46
2  2024-12-05  19.04
1  2025-01-04  19.80
0  2025-02-03  20.82


In [34]:
import pandas as pd
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
from gym import spaces
from datetime import timedelta

class StockEnv(gym.Env):
    def __init__(self, df, N, M, news_events):
        super(StockEnv, self).__init__()
        self.df = df
        self.N = N
        self.M = M
        self.news_events = news_events
        self.current_step = len(df) - 1  # 가장 최근 날짜로 초기화
        self.action_space = spaces.Discrete(3)  # 0: 풀매수, 1: 풀매도, 2: 존버
        self.observation_space = spaces.Box(low=0, high=1, shape=(len(df.columns),), dtype=np.float32)
        
    def reset(self):
        self.current_step = len(self.df) - 1  # 가장 최근 날짜로 초기화
        return self._next_observation()
    
    def _next_observation(self):
        obs = self.df.iloc[self.current_step].values
        return obs
    
    def step(self, action):
        self.current_step -= 1  # 과거로 이동
        
        if self.current_step < 0:
            done = True
        else:
            done = False
        
        reward = self._calculate_reward(action)
        
        obs = self._next_observation()
        
        return obs, reward, done, {}
    
    def _calculate_reward(self, action):
        current_price = self.df.iloc[self.current_step]['종가']
        future_step = self.current_step - (self.N * 12 + self.M)  # N년 M개월 전의 스텝 계산
        if future_step < 0:
            future_price = self.df.iloc[0]['종가']
        else:
            future_price = self.df.iloc[future_step]['종가']
        
        news = self.news_events[self.current_step % len(self.news_events)]  # 현재 스텝의 뉴스 이벤트
        news_factor = 1.3
        if news == 'Good':
            news_factor = 1.8  # 좋은 뉴스일 경우 보상 증가 (1년동안 오른 정도 반영)
        elif news == 'Bad':
            news_factor = 0.7  # 나쁜 뉴스일 경우 보상 감소 (1달 전 최저가 반영)
        
        if action == 0:  # 풀매수 가즈아~
            reward = (future_price - current_price) * news_factor
        elif action == 1:  # 풀매도 가즈아!
            reward = (current_price - future_price) * news_factor
        else:  # 존~~~~~버
            reward = 0
        
        return reward
    
    def render(self, mode='human'):
        pass

# Step 1: 데이터 파일 읽기
file_path = 'GPT-Now.csv'
data = pd.read_csv(file_path)

# 데이터 전처리
data = data[['날짜', '종가']]  # 필요한 컬럼만 선택
data['날짜'] = pd.to_datetime(data['날짜'])
data.sort_values(by='날짜', ascending=True, inplace=True)  # 날짜를 기준으로 정렬
data.set_index('날짜', inplace=True)

# 사용자 입력으로 뉴스 이벤트 리스트 생성
user_news_events = ['Good', 'Good', 'Bad', 'Good', 'Good', 'Good', 'Good']  # 예시 입력
news_events = user_news_events * (len(data) // len(user_news_events)) + user_news_events[:len(data) % len(user_news_events)]

# Step 2: 환경 설정
N = 3  # 년
M = 6  # 개월

# 환경 생성
env = DummyVecEnv([lambda: StockEnv(data, N, M, news_events)])

# 학습된 모델 로드
model = PPO.load("ppo_stock_model.zip")  # 학습된 모델 파일 경로

# 특정 시점의 데이터로 환경 초기화
obs = env.reset()

# N년 M개월 동안의 예측
steps = N * 12 + M

predicted_prices = []
predicted_dates = []

# 시작 날짜
start_date = data.index[-1]

for _ in range(steps):
    action, _ = model.predict(obs)
    obs, rewards, dones, info = env.step([action])
    current_step = env.envs[0].current_step
    current_price = env.envs[0].df.iloc[current_step]['종가']
    current_date = start_date + timedelta(days=(steps - len(predicted_prices)) * 30)  # 매 스텝 30일 추가
    predicted_prices.append(current_price)
    predicted_dates.append(current_date)
    if dones:
        break

predicted_df = pd.DataFrame({
    '날짜': predicted_dates,
    '예측 종가': predicted_prices
})

# 표 형태로 출력
predicted_df = predicted_df.sort_values(by='날짜', ascending=True)  # 날짜 오름차순 정렬
print(predicted_df)

# 예측 결과 저장 (선택 사항)
predicted_df.to_csv('predicted_prices.csv', index=False)


           날짜   예측 종가
41 2024-07-10   87.04
40 2024-08-09   90.62
39 2024-09-08   88.19
38 2024-10-08   86.00
37 2024-11-07   87.42
36 2024-12-07   84.03
35 2025-01-06   84.67
34 2025-02-05   76.20
33 2025-03-07   79.52
32 2025-04-06   82.42
31 2025-05-06   79.68
30 2025-06-05   82.63
29 2025-07-05   87.73
28 2025-08-04   87.76
27 2025-09-03   86.40
26 2025-10-03   83.04
25 2025-11-02   85.82
24 2025-12-02   88.79
23 2026-01-01   92.14
22 2026-01-31   90.55
21 2026-03-02   90.41
20 2026-04-01   88.75
19 2026-05-01   89.88
18 2026-05-31   90.40
17 2026-06-30   91.36
16 2026-07-30   94.63
15 2026-08-29   94.36
14 2026-09-28   92.48
13 2026-10-28   94.78
12 2026-11-27   95.39
11 2026-12-27   94.95
10 2027-01-26  103.80
9  2027-02-25  106.47
8  2027-03-27  113.90
7  2027-04-26  114.83
6  2027-05-26  110.50
5  2027-06-25  109.63
4  2027-07-25  115.00
3  2027-08-24  116.44
2  2027-09-23  122.44
1  2027-10-23  121.00
0  2027-11-22  120.89
